In [257]:
import numpy as np 
import pandas as pd 
import portion as P
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable 
from math import prod


# Transforming LCL:s to linear problems

Goal is to find $\alpha, \beta, a,b,c...$ such that if some $A, A, B$ is possible for active and/or passive nodes, then the corresponding $a+a+b$ will be $\geq \alpha$ and/or $\leq \beta $. Additionally it would be great to see if these values $a, b, c...$ could be expanded to intervals.

## Notes:
It seems to be difficult to find problems that can be expressed in this way, as the constraints for active and passive nodes are unbounded in the other direction. This means that when we have an absolute order for our labels, active (passive) nodes must always allow strengthening (weakening) of labels, as the sum will always be larger (smaller) and thus complying with the constraints. The current version tries to find ranges $\alpha, \beta \subseteq [0, \max(d,\delta)]$, so that sum of edges incident to active (passive) nodes must be within the range $\alpha \; (\beta)$.

## Example problem:

```
A AB AB

B AB AB
```
can be interpreted as $\alpha = 1, \beta=2, A = \left[ 0,\frac{1}{3}\right), B =  \left(\frac{2}{3}, 1\right]$.

What is the system of linear inequalities that could be used to derive this solution? Try:

\begin{align*}
3a &\leq \beta \\
2a+b &\leq \beta \\
a+2b &\leq \beta \\
3b &> \beta \\
\\
3a &< \alpha \\
2a+b &\geq \alpha \\
a+2b &\geq \alpha \\
3b &\geq \alpha

\end{align*}

In [258]:
# Initialize variables

# Add active and passive constraints in RE-formalism
active = """
A B B

"""
passive = """
AB B
"""

# Enable/disable debug mode
debug = False



In [259]:
# Create table of possible neighbourhoods and whether they are suitable for active/passive nodes

actives = list(map(lambda x: x.split(), active.strip().split("\n")))
passives = list(map(lambda x: x.split(), passive.strip().split("\n")))

# max(d, delta)
d = len(actives[0])
delta = len(passives[0])
sum_max = max(d, delta)

variables = sorted(list(set(list("".join("".join([active, passive]).split())))))

combinations = pd.DataFrame({"combination": itertools.combinations_with_replacement(variables, d)})
if d!= delta:
    combinations = combinations.append(pd.DataFrame({"combination": itertools.combinations_with_replacement(variables, delta)})).reset_index(drop=True)

combinations["active"] = False
combinations["passive"] = False

for row in passives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "passive"] = True 

for row in actives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "active"] = True 

if debug: print(combinations)


In [260]:
def create_md_row(solution):
    s = "PROBLEM NAME | $"
    s += " \\\\ ".join([" \\; ".join(row) for row in actives])
    s += "$ | $"
    s += " \\\\ ".join([" \\; ".join(row) for row in passives])
    s += "$ | $"
    alpha = (next(var.value() for var in model.variables() if "alpha_0" in var.name), next(var.value() for var in model.variables() if "alpha_1" in var.name))
    s += f"[{alpha[0]}, {alpha[1]}]"
    s += "$ | $"
    beta = (next(var.value() for var in model.variables() if "beta_0" in var.name), next(var.value() for var in model.variables() if "beta_1" in var.name))
    s += f"[{beta[0]}, {beta[1]}]"
    s += "$ | $"
    s += ", ".join(f"{var.name}: {var.value()}" for  var in solution if "trick" not in var.name and "alpha" not in var.name and "beta" not in var.name) 
    s += "$"
    return s


In [261]:

epsilon = 0.001

model = LpProblem(name="Reductions", sense=LpMaximize)

alpha_0 = LpVariable(name = "alpha_0", lowBound=0, upBound=sum_max)
alpha_1 = LpVariable(name = "alpha_1", lowBound=0, upBound=sum_max)

beta_0 = LpVariable(name = "beta_0", lowBound=0, upBound=sum_max)
beta_1 = LpVariable(name = "beta_1", lowBound=0, upBound=sum_max)

pulp_variables = dict(zip(variables, [LpVariable(name = v, lowBound=0, upBound=1) for v in variables]))

# Try different targets in order to get more readable solutions
model += beta_1 - beta_0 + alpha_1-alpha_0

# Add constraints
trick_variables = []
for index, row in combinations.iterrows():
    if row["active"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=alpha_0, f"{index}_Active_low")
        model += (sum(pulp_variables[c] for c in row["combination"])<=alpha_1, f"{index}_Active_high")
    else:
        # Sum not in [alpha_0, alpha_1] is the same as sum<alpha_0 OR sum>alpha_1
        # Trick to implement OR in linear programming from https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf
        trick_variables.append(LpVariable(name = f"trick_{index}_a", lowBound=0, upBound=1, cat='Binary'))
        model += (sum(pulp_variables[c] for c in row["combination"])>= alpha_1+epsilon - 1000000*trick_variables[-1], f"{index}_not_Active_high")
        model += (sum(pulp_variables[c] for c in row["combination"])<= alpha_0-epsilon + 1000000*(1-trick_variables[-1]), f"{index}_not_Active_low")

    if row["passive"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=beta_0, f"{index}_Passive_low")
        model += (sum(pulp_variables[c] for c in row["combination"])<=beta_1, f"{index}_Passive_high")
    else:
        trick_variables.append(LpVariable(name = f"trick_{index}_p", lowBound=0, upBound=1, cat='Binary'))
        model += (sum(pulp_variables[c] for c in row["combination"])>= beta_1+epsilon - 1000000*trick_variables[-1], f"{index}_not_Passive_high")
        model += (sum(pulp_variables[c] for c in row["combination"])<= beta_0-epsilon + 1000000*(1-trick_variables[-1]), f"{index}_not_Passive_low")
    

    
if debug: print(model)

if model.solve() == 1:
    print("Linear model found:")

    for var in model.variables():
        if "trick" not in var.name or debug:
            print(var, var.value())

    print("\nCopy problem to markdown table:\n", create_md_row(model.variables()), sep="")

else: print("No suitable model found")

Linear model found:
A 1.0
B 0.66666667
alpha_0 2.001
alpha_1 2.6656667
beta_0 0.0
beta_1 1.999

Add problem to markdown table:
PROBLEM NAME | $A \; B \; B$ | $AB \; B$ | $[2.001, 2.6656667]$ | $[0.0, 1.999]$ | $A: 1.0, B: 0.66666667$
